In [1]:
import ee
import geopandas as gpd
import pandas as pd
import json
import os
import numpy as np
from shapely.geometry import shape, mapping
import re
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
ee.Authenticate()
# create a new project
ee.Initialize()

In [3]:
CT_polygon_folder = './merged_irr_final/merged_irr_final/'
CT_polygon = gpd.read_file(CT_polygon_folder + 'merged_irr_final.shp')
CT_polygon

,Submission,deviceid,start,end,uniqueID,collection,country,state,district,subDistric,...,DeviceID_1,Edits,FormVersio,sowing,GID,Area_m2,Irr,Origin,shape_P1,geometry
0,2023/06/19 06:02:07.676,collect:yYr8fHQlZDx2BoZK,2023/06/19 09:35:02.278,2023/06/19 11:27:21.373,2.020000e+14,2023-06-19,India,Bihar,Samastipur,Pusa,...,collect:yYr8fHQlZDx2BoZK,0,21042023.0,09-11-22,10,1125,2,p1,None,"POLYGON ((85.66504 25.90189, 85.66514 25.90254..."
1,2023/06/15 15:46:21.320,collect:oNnFCmaXn9cA1vGn,2023/06/15 12:31:27.640,2023/06/15 20:23:40.825,2.020000e+14,2023-06-15,India,Bihar,Vaishali,Patepur,...,collect:oNnFCmaXn9cA1vGn,0,21042023.0,18-11-22,11,353,3,p1,None,"POLYGON ((85.48432 25.81530, 85.48445 25.81527..."
2,2023/06/15 15:46:20.987,collect:oNnFCmaXn9cA1vGn,2023/06/15 12:09:25.908,2023/06/15 20:22:06.621,2.020000e+14,2023-06-15,India,Bihar,Vaishali,Patepur,...,collect:oNnFCmaXn9cA1vGn,0,21042023.0,16-11-22,12,431,2,p1,None,"POLYGON ((85.48752 25.81600, 85.48749 25.81584..."
3,2023/06/13 04:36:59.322,collect:blavFNh1Ws80z1FF,2023/06/08 07:33:50.367,2023/06/13 10:06:55.565,2.020000e+14,2023-06-08,India,Bihar,Muzaffarpur,Bandra,...,collect:blavFNh1Ws80z1FF,0,21042023.0,21-11-22,13,258,2,p1,None,"POLYGON ((85.63218 26.05732, 85.63246 26.05728..."
4,2023/06/13 04:36:09.336,collect:blavFNh1Ws80z1FF,2023/06/08 07:43:45.254,2023/06/13 10:06:05.882,2.020000e+14,2023-06-08,India,Bihar,Muzaffarpur,Bandra,...,collect:blavFNh1Ws80z1FF,0,21042023.0,22-11-22,14,468,4,p1,None,"POLYGON ((85.63153 26.05660, 85.63164 26.05706..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,None,collect:tABCfvIh2vWGCL65,2023-08-11T13:34:02.763+05:30,2023-08-11T13:38:29.326+05:30,2.023080e+14,None,India,UttarPradesh,Kushinagar,None,...,collect:tABCfvIh2vWGCL65,0,NaN,13-11-22,3137,1055,4,p3,26.730735214602834 84.05049294233322 0.0 0.0;2...,"POLYGON ((84.05059 26.73120, 84.05069 26.73142..."
422,None,collect:tABCfvIh2vWGCL65,2023-08-11T13:49:18.123+05:30,2023-08-11T13:55:20.239+05:30,2.023080e+14,None,India,UttarPradesh,Kushinagar,None,...,collect:tABCfvIh2vWGCL65,0,NaN,12-11-22,3138,2575,3,p3,26.731106825544632 84.05233696103096 0.0 0.0;2...,"POLYGON ((84.05261 26.73075, 84.05289 26.73153..."
423,None,collect:tABCfvIh2vWGCL65,2023-08-11T09:59:08.833+05:30,2023-08-11T10:02:58.349+05:30,2.023080e+14,None,India,UttarPradesh,Kushinagar,None,...,collect:tABCfvIh2vWGCL65,0,NaN,14-11-22,3139,3475,4,p3,26.79407748046505 84.01092331856489 0.0 0.0;26...,"POLYGON ((84.00954 26.79365, 84.00917 26.79374..."
424,None,collect:tABCfvIh2vWGCL65,2023-08-11T09:54:53.400+05:30,2023-08-11T09:59:00.577+05:30,2.023080e+14,None,India,UttarPradesh,Kushinagar,None,...,collect:tABCfvIh2vWGCL65,0,NaN,10-11-22,3140,931,4,p3,26.794834352863322 84.01130151003599 0.0 0.0;2...,"POLYGON ((84.01074 26.79448, 84.01109 26.79440..."


In [4]:
# list of all the properties of the shapefile
CT_polygon.columns

Index(['Submission', 'deviceid', 'start', 'end', 'uniqueID', 'collection',
       'country', 'state', 'district', 'subDistric', 'village', 'fName',
       'plot1', 'plot2', 'shape_P2', 'plot3', 'shape_P3', 'instanceID',
       'instanceNa', 'KEY', 'SubmitterI', 'SubmitterN', 'Attachment',
       'Attachme_1', 'Status', 'ReviewStat', 'DeviceID_1', 'Edits',
       'FormVersio', 'sowing', 'GID', 'Area_m2', 'Irr', 'Origin', 'shape_P1',
       'geometry'],
      dtype='object')

In [5]:
def convert_custom_to_wkt(custom_coords):
    # Split the input string by semicolon into coordinate blocks
    blocks = custom_coords.split("; ")
    # Prepare the new format
    new_coords = []
    for block in blocks:
        parts = block.split()
        lat, lon = parts[0], parts[1]  # First two elements are lat and lon
        # Switch lat and lon back to lon-lat order
        new_coords.append(f"{lon} {lat}")
    return ", ".join(new_coords)

In [6]:
# select the columns that we want to keep
CT_polygon_simple = CT_polygon[['plot1','shape_P1','plot2', 'shape_P2', 'plot3','shape_P3','geometry', 'sowing', 'Origin']]
for index, row in CT_polygon_simple.iterrows():
    # get rid of the format of the geometry
    geometry_value = str(row['geometry'])
    geometry_value = geometry_value.replace('POLYGON ((', '')
    geometry_value = geometry_value.replace('))', '')
    if pd.isna(row['shape_P1']):
        CT_polygon_simple.at[index, 'shape_P1'] = geometry_value
        CT_polygon_simple.at[index, 'shape_P2'] = convert_custom_to_wkt(CT_polygon_simple.at[index, 'shape_P2'])
        CT_polygon_simple.at[index, 'shape_P3'] = convert_custom_to_wkt(CT_polygon_simple.at[index, 'shape_P3'])
    if pd.isna(row['shape_P2']):
        CT_polygon_simple.at[index, 'shape_P2'] = geometry_value
        CT_polygon_simple.at[index, 'shape_P1'] = convert_custom_to_wkt(CT_polygon_simple.at[index, 'shape_P1'])
        CT_polygon_simple.at[index, 'shape_P3'] = convert_custom_to_wkt(CT_polygon_simple.at[index, 'shape_P3'])
    if pd.isna(row['shape_P3']):
        CT_polygon_simple.at[index, 'shape_P3'] = geometry_value
        CT_polygon_simple.at[index, 'shape_P1'] = convert_custom_to_wkt(CT_polygon_simple.at[index, 'shape_P1'])
        CT_polygon_simple.at[index, 'shape_P2'] = convert_custom_to_wkt(CT_polygon_simple.at[index, 'shape_P2'])

CT_polygon_simple

,plot1,shape_P1,plot2,shape_P2,plot3,shape_P3,geometry,sowing,Origin
0,"CT with 2 irrigations (21 DAS, 65 DAS)","85.66504404500006 25.901887871000042, 85.66513...","CT with 2 irrigations (21 DAS, 65 DAS)","85.6649886444211 25.9025527289652, 85.66489845...","CT with 2 irrigations (21 DAS, 65 DAS)","85.66488403826952 25.902564189526103, 85.66478...","POLYGON ((85.66504 25.90189, 85.66514 25.90254...",09-11-22,p1
1,"CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","85.48432007431984 25.815295384789376, 85.48445...","CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","85.48387248069048 25.8155138996796, 85.4841021...","CT with 2 irrigations (21 DAS, 65 DAS)","85.4851559177041 25.81549941251704, 85.4851887...","POLYGON ((85.48432 25.81530, 85.48445 25.81527...",18-11-22,p1
2,"CT with 2 irrigations (21 DAS, 65 DAS)","85.48751693218946 25.81599740771367, 85.487492...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","85.48746731132269 25.81576863199851, 85.487459...","CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","85.48721887171268 25.815675371059076, 85.48702...","POLYGON ((85.48752 25.81600, 85.48749 25.81584...",16-11-22,p1
3,"CT with 2 irrigations (21 DAS, 65 DAS)","85.63218355178833 26.05732443921286, 85.632462...","CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...",85.6321520358324 26.057227453742904,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...",85.63213862478733 26.05707565023748,"POLYGON ((85.63218 26.05732, 85.63246 26.05728...",21-11-22,p1
4,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","85.63152842223644 26.056603973805093, 85.63164...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)",85.63159614801407 26.056586504271113,"CT with 2 irrigations (21 DAS, 65 DAS)",85.63165716826916 26.056580781492205,"POLYGON ((85.63153 26.05660, 85.63164 26.05706...",22-11-22,p1
...,...,...,...,...,...,...,...,...,...
421,"CT with 2 irrigations (21 DAS, 65 DAS)",84.05049294233322 26.730735214602834,"CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)",84.05052479356527 26.73095410886616,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","84.05059151351452 26.731204144889677, 84.05069...","POLYGON ((84.05059 26.73120, 84.05069 26.73142...",13-11-22,p3
422,"CT with 2 irrigations (21 DAS, 65 DAS)",84.05233696103096 26.731106825544632,"CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)",84.05257500708103 26.731043942231278,"CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","84.05261356383562 26.730754977987278, 84.05288...","POLYGON ((84.05261 26.73075, 84.05289 26.73153...",12-11-22,p3
423,"CT with 2 irrigations (21 DAS, 65 DAS)",84.01092331856489 26.79407748046505,"CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)",84.00996442884207 26.793547457286177,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","84.00953963398933 26.79365040946949, 84.009171...","POLYGON ((84.00954 26.79365, 84.00917 26.79374...",14-11-22,p3
424,"CT with 2 irrigations (21 DAS, 65 DAS)",84.01130151003599 26.794834352863322,"CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)",84.01078820228577 26.794711649495973,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","84.01073891669512 26.79447731562329, 84.011085...","POLYGON ((84.01074 26.79448, 84.01109 26.79440...",10-11-22,p3


In [7]:
# sort by shape_P1, add a group column, each group has 3 rows
CT_polygon_simple = CT_polygon_simple.sort_values(by='shape_P1')
# reset the index
CT_polygon_simple = CT_polygon_simple.reset_index(drop=True)
# add a group column
CT_polygon_simple['group'] = CT_polygon_simple.index // 3
# check if for each group, the shape_P1 are the same
# for i in range(0, len(CT_polygon_simple), 3):
#     assert CT_polygon_simple.at[i, 'shape_P1'] == CT_polygon_simple.at[i+1, 'shape_P1']
#     assert CT_polygon_simple.at[i, 'shape_P2'] == CT_polygon_simple.at[i+2, 'shape_P2']
# Though the assertation is not correct, we can still use the group column to group the polygons
CT_polygon_simple

,plot1,shape_P1,plot2,shape_P2,plot3,shape_P3,geometry,sowing,Origin,group
0,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","83.27556654810905 26.51913650693408, 83.275912...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","83.275759331882 26.518783706222347, 83.2762552...","CT with 2 irrigations (21 DAS, 65 DAS)","83.27629443258047 26.51849600483952, 83.276851...","POLYGON ((83.27576 26.51878, 83.27626 26.51856...",21-11-22,p2,0
1,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","83.27556654810905 26.51913650693408, 83.275912...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","83.275759331882 26.518783706222347, 83.2762552...","CT with 2 irrigations (21 DAS, 65 DAS)","83.27629443258047 26.51849600483952, 83.276851...","POLYGON ((83.27629 26.51850, 83.27685 26.51825...",21-11-22,p3,0
2,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","83.27556654810905 26.51913650693408, 83.275912...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","83.275759331882 26.518783706222347, 83.2762552...","CT with 2 irrigations (21 DAS, 65 DAS)","83.27629443258047 26.51849600483952, 83.276851...","POLYGON ((83.27557 26.51914, 83.27591 26.51899...",21-11-22,p1,0
3,"CT with 2 irrigations (21 DAS, 65 DAS)","83.27619284391403 26.519737105652258, 83.27669...","CT with 2 irrigations (21 DAS, 65 DAS)","83.27575396746397 26.5195697063253, 83.2765804...","CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","83.27599234879017 26.519060306872156, 83.27640...","POLYGON ((83.27599 26.51906, 83.27640 26.51889...",21-11-22,p3,1
4,"CT with 2 irrigations (21 DAS, 65 DAS)","83.27619284391403 26.519737105652258, 83.27669...","CT with 2 irrigations (21 DAS, 65 DAS)","83.27575396746397 26.5195697063253, 83.2765804...","CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","83.27599234879017 26.519060306872156, 83.27640...","POLYGON ((83.27575 26.51957, 83.27658 26.51924...",21-11-22,p2,1
...,...,...,...,...,...,...,...,...,...,...
421,"CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)",86.82947404682636 25.019537835150018,"CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","86.82974126189947 25.0194755530532, 86.8298254...","CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...",86.83001685887575 25.019453070727568,"POLYGON ((86.82974 25.01948, 86.82983 25.01976...",27-11-22,p2,140
422,"CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","86.82947404682636 25.019537835150018, 86.82956...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)",86.82974126189947 25.0194755530532,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...",86.83001685887575 25.019453070727568,"POLYGON ((86.82947 25.01954, 86.82956 25.01982...",27-11-22,p1,140
423,"CT with 2 irrigations (21 DAS, 65 DAS)","86.95964741500006 24.99100911000005, 86.959310...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","86.95972044020891 24.991205334930196, 86.95933...","CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","86.95999804884195 24.990927887511265, 86.96003...","POLYGON ((86.95965 24.99101, 86.95931 24.99113...",12-11-22,p1,141
424,"CT with 2 irrigations (21 DAS, 65 DAS)","86.9596641138196 24.99100325116565, 86.9596855...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","86.95972044020891 24.991205334930196, 86.95933...","CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","86.95999804884195 24.990927887511265, 86.96003...","POLYGON ((86.95972 24.99121, 86.95934 24.99132...",12-11-22,p2,141


In [8]:
# check if all the polygons are valid
for index, row in CT_polygon_simple.iterrows():
    if not row['geometry'].is_valid:
        print(index)
        print(row['plot1'])
        print(row['plot2'])
        print(row['plot3'])
        print(row['shape_P1'])
        print(row['shape_P2'])
        print(row['shape_P3'])
        print(row['Origin'])
        print(row['sowing'])
        print(row['geometry'])
        print('-----------------------')

89
CT with 2 irrigations (21 DAS, 65 DAS)
CT with 3 irrigations  (21 DAS, 65 DAS, 105 DAS)
CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS, 105 DAS)
84.0112055 26.7943881, 84.01124812890401 26.794354362235342, 84.01148797081937 26.794923539318066, 84.01244375875075 26.794701596053105, 84.01229341008738 26.79411452031998, 84.0112055 26.7943881
84.0112014 26.7943848
84.01083715260029 26.7935705018237
p1
08-11-22
POLYGON ((84.0112055 26.7943881, 84.01124812890401 26.794354362235342, 84.01148797081937 26.794923539318066, 84.01244375875075 26.794701596053105, 84.01229341008738 26.79411452031998, 84.0112055 26.7943881))
-----------------------
290
CT with 2 irrigations (21 DAS, 65 DAS)
CT with 3 irrigations  (21 DAS, 65 DAS, 105 DAS)
CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS, 105 DAS)
85.54671362042427 26.00735816657806, 85.54683532565832 26.007679077593696, 85.54725542664528 26.007579037821863, 85.54715556401149 26.007292844432335, 85.54671745681834 26.00737534513754, 85.54671362042427 26

In [9]:
# make the geometry valid
for index, row in CT_polygon_simple.iterrows():
    CT_polygon_simple.at[index, 'geometry'] = row['geometry'].buffer(0)
    # check if the geometry is valid
    if not CT_polygon_simple.at[index, 'geometry'].is_valid:
        print('still not valid')

In [10]:
CT_polygon_simple = CT_polygon_simple[CT_polygon_simple['sowing'].notna()]
CT_polygon_simple['sowing_date'] = pd.to_datetime(CT_polygon_simple['sowing'], format='%d-%m-%y')
CT_polygon_simple['sowing_date'] = CT_polygon_simple['sowing_date'].dt.strftime('%Y-%m-%d')
CT_polygon_simple

c:\Users\Shaoy\miniconda3\envs\gee\Lib\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\Shaoy\miniconda3\envs\gee\Lib\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


,plot1,shape_P1,plot2,shape_P2,plot3,shape_P3,geometry,sowing,Origin,group,sowing_date
0,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","83.27556654810905 26.51913650693408, 83.275912...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","83.275759331882 26.518783706222347, 83.2762552...","CT with 2 irrigations (21 DAS, 65 DAS)","83.27629443258047 26.51849600483952, 83.276851...","POLYGON ((83.27576 26.51878, 83.27626 26.51856...",21-11-22,p2,0,2022-11-21
1,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","83.27556654810905 26.51913650693408, 83.275912...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","83.275759331882 26.518783706222347, 83.2762552...","CT with 2 irrigations (21 DAS, 65 DAS)","83.27629443258047 26.51849600483952, 83.276851...","POLYGON ((83.27629 26.51850, 83.27685 26.51825...",21-11-22,p3,0,2022-11-21
2,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","83.27556654810905 26.51913650693408, 83.275912...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","83.275759331882 26.518783706222347, 83.2762552...","CT with 2 irrigations (21 DAS, 65 DAS)","83.27629443258047 26.51849600483952, 83.276851...","POLYGON ((83.27557 26.51914, 83.27591 26.51899...",21-11-22,p1,0,2022-11-21
3,"CT with 2 irrigations (21 DAS, 65 DAS)","83.27619284391403 26.519737105652258, 83.27669...","CT with 2 irrigations (21 DAS, 65 DAS)","83.27575396746397 26.5195697063253, 83.2765804...","CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","83.27599234879017 26.519060306872156, 83.27640...","POLYGON ((83.27599 26.51906, 83.27640 26.51889...",21-11-22,p3,1,2022-11-21
4,"CT with 2 irrigations (21 DAS, 65 DAS)","83.27619284391403 26.519737105652258, 83.27669...","CT with 2 irrigations (21 DAS, 65 DAS)","83.27575396746397 26.5195697063253, 83.2765804...","CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","83.27599234879017 26.519060306872156, 83.27640...","POLYGON ((83.27575 26.51957, 83.27658 26.51924...",21-11-22,p2,1,2022-11-21
...,...,...,...,...,...,...,...,...,...,...,...
421,"CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)",86.82947404682636 25.019537835150018,"CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","86.82974126189947 25.0194755530532, 86.8298254...","CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...",86.83001685887575 25.019453070727568,"POLYGON ((86.82974 25.01948, 86.82983 25.01976...",27-11-22,p2,140,2022-11-27
422,"CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","86.82947404682636 25.019537835150018, 86.82956...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)",86.82974126189947 25.0194755530532,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...",86.83001685887575 25.019453070727568,"POLYGON ((86.82947 25.01954, 86.82956 25.01982...",27-11-22,p1,140,2022-11-27
423,"CT with 2 irrigations (21 DAS, 65 DAS)","86.95964741500006 24.99100911000005, 86.959310...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","86.95972044020891 24.991205334930196, 86.95933...","CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","86.95999804884195 24.990927887511265, 86.96003...","POLYGON ((86.95965 24.99101, 86.95931 24.99113...",12-11-22,p1,141,2022-11-12
424,"CT with 2 irrigations (21 DAS, 65 DAS)","86.9596641138196 24.99100325116565, 86.9596855...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","86.95972044020891 24.991205334930196, 86.95933...","CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","86.95999804884195 24.990927887511265, 86.96003...","POLYGON ((86.95972 24.99121, 86.95934 24.99132...",12-11-22,p2,141,2022-11-12


In [11]:
CT_polygon_simple.to_file(CT_polygon_folder + 'sy_merged.shp')

C:\Users\Shaoy\AppData\Local\Temp\ipykernel_20712\2009034146.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  CT_polygon_simple.to_file(CT_polygon_folder + 'sy_merged.shp')


In [11]:
# extract the sowing date for each records, and get the sentinel-2 image for that date for each polygon

def convert_geom_ee(geom):
    geo_json = mapping(geom)
    return ee.Geometry(geo_json)


CT_polygon_simple['ee_geom'] = CT_polygon_simple['geometry'].apply(convert_geom_ee)

# only keep records with sowing date
CT_polygon_sowing = CT_polygon_simple[CT_polygon_simple['sowing'].notna()]
CT_polygon_sowing

,plot1,shape_P1,plot2,shape_P2,plot3,shape_P3,geometry,sowing,Origin,group,ee_geom
0,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","83.27556654810905 26.51913650693408, 83.275912...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","83.275759331882 26.518783706222347, 83.2762552...","CT with 2 irrigations (21 DAS, 65 DAS)","83.27629443258047 26.51849600483952, 83.276851...","POLYGON ((83.27576 26.51878, 83.27626 26.51856...",21-11-22,p2,0,"ee.Geometry({\n ""functionInvocationValue"": {\..."
1,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","83.27556654810905 26.51913650693408, 83.275912...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","83.275759331882 26.518783706222347, 83.2762552...","CT with 2 irrigations (21 DAS, 65 DAS)","83.27629443258047 26.51849600483952, 83.276851...","POLYGON ((83.27629 26.51850, 83.27685 26.51825...",21-11-22,p3,0,"ee.Geometry({\n ""functionInvocationValue"": {\..."
2,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","83.27556654810905 26.51913650693408, 83.275912...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","83.275759331882 26.518783706222347, 83.2762552...","CT with 2 irrigations (21 DAS, 65 DAS)","83.27629443258047 26.51849600483952, 83.276851...","POLYGON ((83.27557 26.51914, 83.27591 26.51899...",21-11-22,p1,0,"ee.Geometry({\n ""functionInvocationValue"": {\..."
3,"CT with 2 irrigations (21 DAS, 65 DAS)","83.27619284391403 26.519737105652258, 83.27669...","CT with 2 irrigations (21 DAS, 65 DAS)","83.27575396746397 26.5195697063253, 83.2765804...","CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","83.27599234879017 26.519060306872156, 83.27640...","POLYGON ((83.27599 26.51906, 83.27640 26.51889...",21-11-22,p3,1,"ee.Geometry({\n ""functionInvocationValue"": {\..."
4,"CT with 2 irrigations (21 DAS, 65 DAS)","83.27619284391403 26.519737105652258, 83.27669...","CT with 2 irrigations (21 DAS, 65 DAS)","83.27575396746397 26.5195697063253, 83.2765804...","CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","83.27599234879017 26.519060306872156, 83.27640...","POLYGON ((83.27575 26.51957, 83.27658 26.51924...",21-11-22,p2,1,"ee.Geometry({\n ""functionInvocationValue"": {\..."
...,...,...,...,...,...,...,...,...,...,...,...
421,"CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)",86.82947404682636 25.019537835150018,"CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","86.82974126189947 25.0194755530532, 86.8298254...","CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...",86.83001685887575 25.019453070727568,"POLYGON ((86.82974 25.01948, 86.82983 25.01976...",27-11-22,p2,140,"ee.Geometry({\n ""functionInvocationValue"": {\..."
422,"CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","86.82947404682636 25.019537835150018, 86.82956...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)",86.82974126189947 25.0194755530532,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...",86.83001685887575 25.019453070727568,"POLYGON ((86.82947 25.01954, 86.82956 25.01982...",27-11-22,p1,140,"ee.Geometry({\n ""functionInvocationValue"": {\..."
423,"CT with 2 irrigations (21 DAS, 65 DAS)","86.95964741500006 24.99100911000005, 86.959310...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","86.95972044020891 24.991205334930196, 86.95933...","CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","86.95999804884195 24.990927887511265, 86.96003...","POLYGON ((86.95965 24.99101, 86.95931 24.99113...",12-11-22,p1,141,"ee.Geometry({\n ""functionInvocationValue"": {\..."
424,"CT with 2 irrigations (21 DAS, 65 DAS)","86.9596641138196 24.99100325116565, 86.9596855...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","86.95972044020891 24.991205334930196, 86.95933...","CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","86.95999804884195 24.990927887511265, 86.96003...","POLYGON ((86.95972 24.99121, 86.95934 24.99132...",12-11-22,p2,141,"ee.Geometry({\n ""functionInvocationValue"": {\..."


In [12]:
def get_sentinel_image(ee_geometry, image_date):
    # get the sowing date
    # image_date = pd.to_datetime(image_date, format='%d-%m-%y').strftime('%Y-%m-%d')
    # convert date to ee date
    image_date = ee.Date(image_date)
    start_date = image_date.advance(-3, 'day')
    end_date = image_date.advance(8, 'day') # revisit date for sentinel-2 is 10 days
    # get the sentinel-2 image
    sentinel2 = ee.ImageCollection('COPERNICUS/S2').filterBounds(ee_geometry).filterDate(start_date, end_date).first()
    sentinel2 = sentinel2.clip(ee_geometry)
    return sentinel2

In [13]:
for index, row in CT_polygon_sowing.iterrows():
    sentinel2 = get_sentinel_image(row['ee_geom'], row['sowing_date'])
    # print(sentinel2.getInfo())
    if sentinel2 is None:
        print('No sentinel-2 image found for the following record: Group: {}, Plot:{}, Sowing:{}'.format(row['group'], row['Origin'], row['sowing']))
        continue
    red = sentinel2.select('B4')
    nir = sentinel2.select('B8')
    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
    
    CT_polygon_sowing.at[index, 'sentinel2_sowing_date'] = sentinel2
    CT_polygon_sowing.at[index, 'ndvi'] = ndvi
    
    
CT_polygon_sowing

c:\Users\Shaoy\miniconda3\envs\gee\Lib\site-packages\ee\deprecation.py:202: DeprecationWarning: 

Attention required for COPERNICUS/S2! You are using a deprecated asset.
To ensure continued functionality, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2

  warnings.warn(warning, category=DeprecationWarning)
c:\Users\Shaoy\miniconda3\envs\gee\Lib\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\Shaoy\miniconda3\envs\gee\Lib\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

,plot1,shape_P1,plot2,shape_P2,plot3,shape_P3,geometry,sowing,Origin,group,ee_geom,sentinel2_sowing_date,ndvi
0,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","83.27556654810905 26.51913650693408, 83.275912...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","83.275759331882 26.518783706222347, 83.2762552...","CT with 2 irrigations (21 DAS, 65 DAS)","83.27629443258047 26.51849600483952, 83.276851...","POLYGON ((83.27576 26.51878, 83.27626 26.51856...",21-11-22,p2,0,"ee.Geometry({\n ""functionInvocationValue"": {\...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ..."
1,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","83.27556654810905 26.51913650693408, 83.275912...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","83.275759331882 26.518783706222347, 83.2762552...","CT with 2 irrigations (21 DAS, 65 DAS)","83.27629443258047 26.51849600483952, 83.276851...","POLYGON ((83.27629 26.51850, 83.27685 26.51825...",21-11-22,p3,0,"ee.Geometry({\n ""functionInvocationValue"": {\...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ..."
2,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","83.27556654810905 26.51913650693408, 83.275912...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","83.275759331882 26.518783706222347, 83.2762552...","CT with 2 irrigations (21 DAS, 65 DAS)","83.27629443258047 26.51849600483952, 83.276851...","POLYGON ((83.27557 26.51914, 83.27591 26.51899...",21-11-22,p1,0,"ee.Geometry({\n ""functionInvocationValue"": {\...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ..."
3,"CT with 2 irrigations (21 DAS, 65 DAS)","83.27619284391403 26.519737105652258, 83.27669...","CT with 2 irrigations (21 DAS, 65 DAS)","83.27575396746397 26.5195697063253, 83.2765804...","CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","83.27599234879017 26.519060306872156, 83.27640...","POLYGON ((83.27599 26.51906, 83.27640 26.51889...",21-11-22,p3,1,"ee.Geometry({\n ""functionInvocationValue"": {\...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ..."
4,"CT with 2 irrigations (21 DAS, 65 DAS)","83.27619284391403 26.519737105652258, 83.27669...","CT with 2 irrigations (21 DAS, 65 DAS)","83.27575396746397 26.5195697063253, 83.2765804...","CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","83.27599234879017 26.519060306872156, 83.27640...","POLYGON ((83.27575 26.51957, 83.27658 26.51924...",21-11-22,p2,1,"ee.Geometry({\n ""functionInvocationValue"": {\...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,"CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)",86.82947404682636 25.019537835150018,"CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","86.82974126189947 25.0194755530532, 86.8298254...","CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...",86.83001685887575 25.019453070727568,"POLYGON ((86.82974 25.01948, 86.82983 25.01976...",27-11-22,p2,140,"ee.Geometry({\n ""functionInvocationValue"": {\...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ..."
422,"CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","86.82947404682636 25.019537835150018, 86.82956...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)",86.82974126189947 25.0194755530532,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...",86.83001685887575 25.019453070727568,"POLYGON ((86.82947 25.01954, 86.82956 25.01982...",27-11-22,p1,140,"ee.Geometry({\n ""functionInvocationValue"": {\...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ..."
423,"CT with 2 irrigations (21 DAS, 65 DAS)","86.95964741500006 24.99100911000005, 86.959310...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","86.95972044020891 24.991205334930196, 86.95933...","CT with 4 irrigation

In [14]:
# if we can access ndvi.getinfo(), then we can calculate the mean ndvi for each polygon
for index, row in CT_polygon_sowing.iterrows():
    # only if ndvi.getinfo() does not return an error, we can calculate the mean ndvi
    try:
        ndvi = row['ndvi']
        mean_ndvi = ndvi.reduceRegion(reducer=ee.Reducer.mean(), geometry=row['ee_geom'], scale=10)
        mean_ndvi = mean_ndvi.getInfo()
        # print(mean_ndvi['NDVI'])
        CT_polygon_sowing.at[index, 'mean_ndvi_at_sowing'] = mean_ndvi['NDVI']
    except Exception as e:
        print(e)
        print('Error in calculating mean ndvi for the following record: Group: {}, Plot:{}, Sowing:{}'.format(row['group'], row['Origin'], row['sowing']))
        continue
    
# CT_polygon_sowing

c:\Users\Shaoy\miniconda3\envs\gee\Lib\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [15]:
CT_polygon_sowing.drop(columns=['ee_geom', 'sentinel2_sowing_date', 'ndvi']).to_file(CT_polygon_folder + 'sy_merged_sowing.shp')
# CT_polygon_sowing.drop(columns=['ndvi']).to_file(CT_polygon_folder + 'sy_merged_sowing.shp')

C:\Users\Shaoy\AppData\Local\Temp\ipykernel_25320\3124968461.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  CT_polygon_sowing.drop(columns=['ee_geom', 'sentinel2_sowing_date', 'ndvi']).to_file(CT_polygon_folder + 'sy_merged_sowing.shp')


In [16]:
# A function to extract the plot irrigation dates for p1, p2, p3 columns
def extract_DAS(row, plot_column):
    das_values = map(int, re.findall(r'(\d+) DAS', row[plot_column]))
    sowing_date = row['sowing_date']
    irr_dates = [sowing_date + pd.DateOffset(days=das) for das in das_values]
    irr_dates = [date.strftime('%d-%m-%y') for date in irr_dates]
    return irr_dates

In [19]:
for index, row in CT_polygon_sowing.iterrows():
    if row['Origin'] == 'p1':
        irr_dates1 = extract_DAS(row, 'plot1')
        CT_polygon_sowing.at[index, 'irr_dates'] = irr_dates1
    elif row['Origin'] == 'p2':
        irr_dates2 = extract_DAS(row, 'plot2')
        CT_polygon_sowing.at[index, 'irr_dates'] = irr_dates2
    elif row['Origin'] == 'p3':
        irr_dates3 = extract_DAS(row, 'plot3')
        CT_polygon_sowing.at[index, 'irr_dates'] = irr_dates3
        

CT_polygon_sowing

,plot1,shape_P1,plot2,shape_P2,plot3,shape_P3,geometry,sowing,Origin,group,ee_geom,sentinel2_sowing_date,ndvi,mean_ndvi_at_sowing,irr_dates
0,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","83.27556654810905 26.51913650693408, 83.275912...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","83.275759331882 26.518783706222347, 83.2762552...","CT with 2 irrigations (21 DAS, 65 DAS)","83.27629443258047 26.51849600483952, 83.276851...","POLYGON ((83.27576 26.51878, 83.27626 26.51856...",21-11-22,p2,0,"ee.Geometry({\n ""functionInvocationValue"": {\...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",0.070139,"[12-12-22, 25-01-23, 06-03-23]"
1,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","83.27556654810905 26.51913650693408, 83.275912...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","83.275759331882 26.518783706222347, 83.2762552...","CT with 2 irrigations (21 DAS, 65 DAS)","83.27629443258047 26.51849600483952, 83.276851...","POLYGON ((83.27629 26.51850, 83.27685 26.51825...",21-11-22,p3,0,"ee.Geometry({\n ""functionInvocationValue"": {\...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",0.078841,"[12-12-22, 25-01-23]"
2,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","83.27556654810905 26.51913650693408, 83.275912...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","83.275759331882 26.518783706222347, 83.2762552...","CT with 2 irrigations (21 DAS, 65 DAS)","83.27629443258047 26.51849600483952, 83.276851...","POLYGON ((83.27557 26.51914, 83.27591 26.51899...",21-11-22,p1,0,"ee.Geometry({\n ""functionInvocationValue"": {\...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",0.066583,"[12-12-22, 25-01-23, 14-02-23, 06-03-23]"
3,"CT with 2 irrigations (21 DAS, 65 DAS)","83.27619284391403 26.519737105652258, 83.27669...","CT with 2 irrigations (21 DAS, 65 DAS)","83.27575396746397 26.5195697063253, 83.2765804...","CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","83.27599234879017 26.519060306872156, 83.27640...","POLYGON ((83.27599 26.51906, 83.27640 26.51889...",21-11-22,p3,1,"ee.Geometry({\n ""functionInvocationValue"": {\...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",0.073121,"[12-12-22, 25-01-23, 14-02-23, 06-03-23]"
4,"CT with 2 irrigations (21 DAS, 65 DAS)","83.27619284391403 26.519737105652258, 83.27669...","CT with 2 irrigations (21 DAS, 65 DAS)","83.27575396746397 26.5195697063253, 83.2765804...","CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","83.27599234879017 26.519060306872156, 83.27640...","POLYGON ((83.27575 26.51957, 83.27658 26.51924...",21-11-22,p2,1,"ee.Geometry({\n ""functionInvocationValue"": {\...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",0.072981,"[12-12-22, 25-01-23]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,"CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)",86.82947404682636 25.019537835150018,"CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","86.82974126189947 25.0194755530532, 86.8298254...","CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...",86.83001685887575 25.019453070727568,"POLYGON ((86.82974 25.01948, 86.82983 25.01976...",27-11-22,p2,140,"ee.Geometry({\n ""functionInvocationValue"": {\...","ee.Image({\n ""functionInvocationValue"": {\n ...","ee.Image({\n ""functionInvocationValue"": {\n ...",0.067926,"[18-12-22, 31-01-23, 12-03-23]"
422,"CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","86.82947404682636 25.019537835150018, 86.82956...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)",86.82974126189947 25.0194755530532,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...",86.83001685887575 25.019453070727568,"POLYGON ((86.82947 25.01954, 86.82956 25.01982...",27-11-22,p1,140,"ee.Geometry({\n ""functionInvocationValue"": {\...","ee.Image({\n ""functionInvocation

In [20]:
# iterate through the irrigation dates, and get the mean ndvi for each date for each polygon
for index, row in CT_polygon_sowing.iterrows():
    for irr_date in row['irr_dates']:
        sentinel2 = get_sentinel_image(row['ee_geom'], irr_date)
        red = sentinel2.select('B4')
        nir = sentinel2.select('B8')
        ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
        mean_ndvi = ndvi.reduceRegion(reducer=ee.Reducer.mean(), geometry=row['ee_geom'], scale=10)
        mean_ndvi = mean_ndvi.getInfo()['NDVI']
        # append mean_ndvi to a new column
        CT_polygon_sowing.at[index, f'{irr_date} mean ndvi'] = mean_ndvi
    
CT_polygon_sowing   

c:\Users\Shaoy\miniconda3\envs\gee\Lib\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\Shaoy\miniconda3\envs\gee\Lib\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
c:\Users\Shaoy\miniconda3\envs\gee\Lib\site-packages\geopandas\geodataframe.py:1543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

,plot1,shape_P1,plot2,shape_P2,plot3,shape_P3,geometry,sowing,Origin,group,...,27-01-23 mean ndvi,30-11-22 mean ndvi,26-11-22 mean ndvi,09-01-23 mean ndvi,23-12-22 mean ndvi,17-03-23 mean ndvi,28-11-22 mean ndvi,11-01-23 mean ndvi,31-01-23 mean ndvi,18-12-22 mean ndvi
0,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","83.27556654810905 26.51913650693408, 83.275912...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","83.275759331882 26.518783706222347, 83.2762552...","CT with 2 irrigations (21 DAS, 65 DAS)","83.27629443258047 26.51849600483952, 83.276851...","POLYGON ((83.27576 26.51878, 83.27626 26.51856...",21-11-22,p2,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","83.27556654810905 26.51913650693408, 83.275912...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","83.275759331882 26.518783706222347, 83.2762552...","CT with 2 irrigations (21 DAS, 65 DAS)","83.27629443258047 26.51849600483952, 83.276851...","POLYGON ((83.27629 26.51850, 83.27685 26.51825...",21-11-22,p3,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","83.27556654810905 26.51913650693408, 83.275912...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","83.275759331882 26.518783706222347, 83.2762552...","CT with 2 irrigations (21 DAS, 65 DAS)","83.27629443258047 26.51849600483952, 83.276851...","POLYGON ((83.27557 26.51914, 83.27591 26.51899...",21-11-22,p1,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"CT with 2 irrigations (21 DAS, 65 DAS)","83.27619284391403 26.519737105652258, 83.27669...","CT with 2 irrigations (21 DAS, 65 DAS)","83.27575396746397 26.5195697063253, 83.2765804...","CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","83.27599234879017 26.519060306872156, 83.27640...","POLYGON ((83.27599 26.51906, 83.27640 26.51889...",21-11-22,p3,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"CT with 2 irrigations (21 DAS, 65 DAS)","83.27619284391403 26.519737105652258, 83.27669...","CT with 2 irrigations (21 DAS, 65 DAS)","83.27575396746397 26.5195697063253, 83.2765804...","CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","83.27599234879017 26.519060306872156, 83.27640...","POLYGON ((83.27575 26.51957, 83.27658 26.51924...",21-11-22,p2,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
421,"CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)",86.82947404682636 25.019537835150018,"CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","86.82974126189947 25.0194755530532, 86.8298254...","CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...",86.83001685887575 25.019453070727568,"POLYGON ((86.82974 25.01948, 86.82983 25.01976...",27-11-22,p2,140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.283849,0.063284
422,"CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","86.82947404682636 25.019537835150018, 86.82956...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)",86.82974126189947 25.0194755530532,"CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...",86.83001685887575 25.019453070727568,"POLYGON ((86.82947 25.01954, 86.82956 25.01982...",27-11-22,p1,140,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.295125,0.063187
423,"CT with 2 irrigations (21 DAS, 65 DAS)","86.95964741500006 24.99100911000005, 86.959310...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","86.95972044020891 24.991205334930196, 86.95933...","CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","86.95999804884195 24.990927887511265, 86.96003...","POLYGON ((86.95965 24.99101, 86.95931 24.99113...",12-11-22,p1,141,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
424,"CT with 2 irrigations (21 DAS, 65 DAS)","86.9596641138196 24.99100325116565, 86.9596855...","CT with 3 irrigations (21 DAS, 65 DAS, 105 DAS)","86.95972044020891 24.991205334930196, 86.95933...","CT with 4 irrigations (21 DAS, 65 DAS, 85 DAS,...","86.95999804884195 24.990927887511265, 86.96003...","POLYGON ((86.95972 24.99121, 86.95934 24.99132...",12-11-22,p2,141,...,NaN,NaN,Na

In [21]:
# check if the mean ndvi for each irrigation date is calculated correctly
# row = CT_polygon_sowing.iloc[0]
# for irr_date in row['irr_dates']:
#     if not pd.isna(row[f'{irr_date} mean ndvi']):
#         print(row[f'{irr_date} mean ndvi'])

In [22]:
# list all the columns
CT_polygon_sowing.columns.values

array(['plot1', 'shape_P1', 'plot2', 'shape_P2', 'plot3', 'shape_P3',
       'geometry', 'sowing', 'Origin', 'group', 'ee_geom',
       'sentinel2_sowing_date', 'ndvi', 'mean_ndvi_at_sowing',
       'irr_dates', '12-12-22 mean ndvi', '25-01-23 mean ndvi',
       '06-03-23 mean ndvi', '14-02-23 mean ndvi', '19-12-22 mean ndvi',
       '01-02-23 mean ndvi', '13-03-23 mean ndvi', '21-02-23 mean ndvi',
       '07-12-22 mean ndvi', '20-01-23 mean ndvi', '09-02-23 mean ndvi',
       '01-03-23 mean ndvi', '21-12-22 mean ndvi', '03-02-23 mean ndvi',
       '15-03-23 mean ndvi', '23-02-23 mean ndvi', '13-12-22 mean ndvi',
       '26-01-23 mean ndvi', '15-02-23 mean ndvi', '07-03-23 mean ndvi',
       '08-12-22 mean ndvi', '21-01-23 mean ndvi', '02-03-23 mean ndvi',
       '10-02-23 mean ndvi', '16-12-22 mean ndvi', '29-01-23 mean ndvi',
       '18-02-23 mean ndvi', '10-03-23 mean ndvi', '01-12-22 mean ndvi',
       '14-01-23 mean ndvi', '30-12-22 mean ndvi', '12-02-23 mean ndvi',
       '04-03-

In [25]:
# drop all ee column and convert the dataframe to a csv file
CT_polygon_ndvi = CT_polygon_sowing.drop(columns=['ee_geom', 'sentinel2_sowing_date', 'ndvi',])
CT_polygon_ndvi.columns.values

array(['plot1', 'shape_P1', 'plot2', 'shape_P2', 'plot3', 'shape_P3',
       'geometry', 'sowing', 'Origin', 'group', 'mean_ndvi_at_sowing',
       'irr_dates', '12-12-22 mean ndvi', '25-01-23 mean ndvi',
       '06-03-23 mean ndvi', '14-02-23 mean ndvi', '19-12-22 mean ndvi',
       '01-02-23 mean ndvi', '13-03-23 mean ndvi', '21-02-23 mean ndvi',
       '07-12-22 mean ndvi', '20-01-23 mean ndvi', '09-02-23 mean ndvi',
       '01-03-23 mean ndvi', '21-12-22 mean ndvi', '03-02-23 mean ndvi',
       '15-03-23 mean ndvi', '23-02-23 mean ndvi', '13-12-22 mean ndvi',
       '26-01-23 mean ndvi', '15-02-23 mean ndvi', '07-03-23 mean ndvi',
       '08-12-22 mean ndvi', '21-01-23 mean ndvi', '02-03-23 mean ndvi',
       '10-02-23 mean ndvi', '16-12-22 mean ndvi', '29-01-23 mean ndvi',
       '18-02-23 mean ndvi', '10-03-23 mean ndvi', '01-12-22 mean ndvi',
       '14-01-23 mean ndvi', '30-12-22 mean ndvi', '12-02-23 mean ndvi',
       '04-03-23 mean ndvi', '24-03-23 mean ndvi', '06-12-22 mean

In [26]:
# arrange sowing date NDVI as other mean NDVI columns
for index, row in CT_polygon_ndvi.iterrows():
    sowing_date = row['sowing']
    CT_polygon_ndvi.at[index, f'{sowing_date} mean ndvi'] = row['mean_ndvi_at_sowing']

c:\Users\Shaoy\miniconda3\envs\gee\Lib\site-packages\geopandas\geodataframe.py:1543: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
c:\Users\Shaoy\miniconda3\envs\gee\Lib\site-packages\geopandas\geodataframe.py:1543: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  super().__setitem__(key, value)
c:\Users\Shaoy\miniconda3\envs\gee\Lib\site-packages\geopandas\geodataframe.py:1543: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which

In [27]:
CT_polygon_ndvi.to_csv(CT_polygon_folder + 'sy_merged_ndvi.csv', index=False)